# Array Electrophysiology Analysis for the DataJoint Workflow: How to Run Locally

This notebook provides a comprehensive guide on populating the ephys files with SpikeInterface using the `array-ephys` pipeline computations. It is designed to be run on a machine with access to the relevant data files for the selected session.

**_Note:_**

- The examples in this notebook use a sample dataset. Replace these entries with your actual database entries to access and analyze your data.



### **Key Steps**

- **Setup**

- **Step 1: Select Session of Interest**

- **Step 2: Populate `Ephys` Schema**


#### **Setup**


First, import the necessary packages for the data pipeline and essential schemas.


In [2]:
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

In [3]:
import datajoint as dj
import numpy as np
import matplotlib.pyplot as plt
import datetime as datetime

In [4]:
from workflow.pipeline import culture, ephys, ephys_sorter

[2024-07-24 23:03:47,215][INFO]: Connecting milagros@db.datajoint.com:3306
[2024-07-24 23:03:48,771][INFO]: Connected milagros@db.datajoint.com:3306


#### **Step 1: Select Session of Interest**


In [5]:
session_key = {
    "organoid_id": "O09",
    "experiment_start_time": datetime.datetime(2023, 5, 18, 12, 25),
    "start_time": "2023-05-18 12:25:00",
    "end_time": "2023-05-18 12:26:30",
}

In [7]:
ephys.EphysSession * ephys.EphysSessionProbe & session_key

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,session_type,probe unique identifier for this model of probe (e.g. serial number),port_id,"used_electrodes list of electrode IDs used in this session (if null, all electrodes are used)"
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:26:30,spike_sorting,Q983,A,=BLOB=


#### **Step 2: Populate `Ephys` Schema**


In [9]:
ephys_key = {**session_key, "insertion_number": 0}

In [10]:
ephys.EphysSessionInfo.heading

# Store header information from the first session file.
organoid_id          : varchar(4)                   # e.g. O17
experiment_start_time : datetime                     # 
insertion_number     : tinyint unsigned             # 
start_time           : datetime                     # 
end_time             : datetime                     # 
---
session_info         : longblob                     # Session header info from intan .rhd file. Get this from the first session file.

In [11]:
ephys.EphysSessionInfo.populate(ephys_key)

In [12]:
ephys.EphysSessionInfo & ephys_key

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,session_info Session header info from intan .rhd file. Get this from the first session file.
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:26:30,=BLOB=


In [14]:
ephys.ClusteringParamSet()

paramset_idx,clustering_method,paramset_desc,param_set_hash,params dictionary of all applicable parameters
0,spykingcircus2,Default parameters for spyking circus2 using SpikeInterface v0.100.1,b6fb9ec2-768c-66b0-2b71-9b8ac91e94da,=BLOB=
1,spykingcircus2,Default parameter set for spyking circus2 using SpikeInterface v0.101.*,434894d0-eb7b-db6c-80e6-638a1322c568,=BLOB=
2,kilosort2,kilosort2 with SpikeInterface version 0.101+,79a731f3-f1b6-c110-5f8a-e25227464de7,=BLOB=
101,spykingcircus2,Spyking circus2 using SpikeInterface v0.101.* and `include_multi_channel_metrics=False`,fd4eb67f-5784-a6ae-6cd8-25a429cad653,=BLOB=


In [15]:
ephys.ClusteringTask & ephys_key & "paramset_idx=101"

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,paramset_idx,clustering_output_dir clustering output directory relative to the clustering root data directory
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:26:30,101,


In [16]:
task_key = (ephys.ClusteringTask & ephys_key & "paramset_idx=101").fetch1("KEY")
task_key

{'organoid_id': 'O09',
 'experiment_start_time': datetime.datetime(2023, 5, 18, 12, 25),
 'insertion_number': 0,
 'start_time': datetime.datetime(2023, 5, 18, 12, 25),
 'end_time': datetime.datetime(2023, 5, 18, 12, 26, 30),
 'paramset_idx': 101}

In [17]:
ephys_sorter.PreProcessing.populate(task_key)

[2024-07-24 20:47:59,156][INFO]: /Users/milagros/Documents/data/organoids/outbox/O09-12_raw/202305181225_202305181226/O09/spykingcircus2_101 created!


In [18]:
ephys_sorter.PreProcessing & task_key

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,paramset_idx,execution_time datetime of the start of this step,execution_duration execution duration in hours
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:26:30,101,2024-07-24 19:47:58,0.00165199


In [19]:
ephys_sorter.SIClustering.populate(task_key)

[2024-07-24 20:54:25,855][INFO]: No existing results found, calling '_run_sorter'


Motion correction activated (probe geometry compatible)


/Users/milagros/miniconda/envs/utah_organoids/lib/python3.10/site-packages/spikeinterface/core/job_tools.py:103: UserWarning: `n_jobs` is not set so parallel processing is disabled! To speed up computations, it is recommended to set n_jobs either globally (with the `spikeinterface.set_global_job_kwargs()` function) or locally (with the `n_jobs` argument). Use `spikeinterface.set_global_job_kwargs?` for more information about job_kwargs.
  warnings.warn(


detect and localize:   0%|          | 0/120 [00:00<?, ?it/s]

/Users/milagros/miniconda/envs/utah_organoids/lib/python3.10/site-packages/spikeinterface/core/baserecordingsnippets.py:264: UserWarning: There is no Probe attached to this recording. Creating a dummy one with contact positions
  warn("There is no Probe attached to this recording. Creating a dummy one with contact positions")


...

estimate_templates:   0%|          | 0/120 [00:00<?, ?it/s]

find spikes (circus-omp-svd):   0%|          | 0/1200 [00:00<?, ?it/s]

We found 112749 spikes


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Final merging, keeping 32 units
spykingcircus2 run time 91.53s


In [20]:
ephys_sorter.SIClustering & task_key

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,paramset_idx,execution_time datetime of the start of this step,execution_duration execution duration in hours
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:26:30,101,2024-07-24 19:54:24,0.0263145


In [22]:
# This is the output directory of the spike sorting
ephys.ClusteringTask & ephys_key & "paramset_idx=101"

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,paramset_idx,clustering_output_dir clustering output directory relative to the clustering root data directory
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:26:30,101,O09-12_raw/202305181225_202305181226/O09/spykingcircus2_101


In [23]:
ephys_sorter.PostProcessing.populate(task_key)

[2024-07-24 20:57:33,398][INFO]: No existing results found, calling '_sorting_analyzer_compute'


estimate_sparsity:   0%|          | 0/120 [00:00<?, ?it/s]

compute_waveforms:   0%|          | 0/120 [00:00<?, ?it/s]

Fitting PCA:   0%|          | 0/32 [00:00<?, ?it/s]

Projecting waveforms:   0%|          | 0/32 [00:00<?, ?it/s]

...

Compute : spike_amplitudes + spike_locations:   0%|          | 0/120 [00:00<?, ?it/s]

calculate_pc_metrics:   0%|          | 0/32 [00:00<?, ?it/s]

write_binary_recording:   0%|          | 0/120 [00:00<?, ?it/s]

extract PCs:   0%|          | 0/120 [00:00<?, ?it/s]

Run:
phy template-gui  /Users/milagros/Documents/data/organoids/outbox/O09-12_raw/202305181225_202305181226/O09/spykingcircus2_101/spykingcircus2/sorting_analyzer/phy/params.py


In [24]:
ephys_sorter.PostProcessing & task_key

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,paramset_idx,execution_time datetime of the start of this step,execution_duration execution duration in hours
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:26:30,101,2024-07-24 19:57:32,0.0359366


In [ ]:
ephys.CuratedClustering.populate(task_key)

In [ ]:
ephys.CuratedClustering & task_key

In [ ]:
ephys.WaveformSet.populate(task_key)

In [ ]:
ephys.WaveformSet & task_key

In [ ]:
ephys.QualityMetrics.populate(task_key)

In [ ]:
ephys.QualityMetrics & task_key